In [5]:
pip install python-docx

In [6]:
pip install openai

In [7]:
pip install openai==0.28

In [4]:
openai.api_key= #'

NameError: name 'openai' is not defined

In [1]:
from docx import Document
import openai
import os
import json

# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []

    # Extract text from paragraphs
    for para in doc.paragraphs:
        full_text.append(para.text)

    # Extract text from tables
    for table in doc.tables:
        for row in table.rows:
            row_text = [cell.text for cell in row.cells]
            full_text.append(' | '.join(row_text))

    return '\n'.join(full_text)

def extract_and_map_fields(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Extract and map the key fields text for an insurance quote:"},
            {"role": "user", "content": text}
        ],
        max_tokens=1500,
        temperature=0.5,
    )
    mapped_fields = response.choices[0].message['content'].strip()
    return mapped_fields

def write_summary(file_path, summary):
    with open(file_path, 'w') as file:
        file.write(summary)

if __name__ == "__main__":
    input_file = '/content/IBP_Problemstatement.docx'
    output_file = 'mapped_fields.json'

    # Read the document
    openai.api_key= '###'
    document_text = read_docx(input_file)

    # Extract and map fields using GPT
    mapped_fields = extract_and_map_fields(document_text)

    # Write the mapped fields to a new file
    write_summary(output_file, mapped_fields)

    print(f"Mapped fields written to {output_file}")


Mapped fields written to mapped_fields.json


In [18]:
import pandas as pd
import openai
import json
import re

# Set up your OpenAI API key
openai.api_key = 'sk-proj-3FVexE0ohPtyBB7t4An8T3BlbkFJLCrIA8pHw4W4lqW4cg2t'

# Load the text file
file_path = '/content/mapped_fields.json'

# Read the file contents
with open(file_path, 'r') as file:
    file_contents = file.read()

# Split the content by sections
sections = file_contents.split("\n\n")

# Initialize a list to store the section data
data = []

# Process each section to extract the title and content
for section in sections:
    if section.strip():  # Ensure the section is not empty
        lines = section.split("\n")
        title = lines[0].strip()
        content = " ".join(lines[1:]).strip()
        data.append({'Section': title, 'Content': content})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Function to generate conditions using GPT-3.5-turbo
def generate_conditions(text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Generate a condition statement for an insurance quote based on the following text:"},
            {"role": "user", "content": text}
        ],
        max_tokens=1500,
        temperature=0.5,
    )
    mapped_fields = response['choices'][0]['message']['content'].strip()
    return mapped_fields

# Function to generate if-else statements using GPT-3.5-turbo
def generate_if_else_statement(condition):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "system", "content": "Assume you are a python expert. Convert the following condition statement into a properly formatted if-elif statement in Python.Also please do not use nested if else statements.Use and statements instead of it.I only want if and elif(strictly you can use elif for creating the else statement at no cost bring an else statement) statemnts. I want all the if else statements in the format if (insurance_policy == "" or insurance_policy == "" or insurance_policy == "" else: insurance_quote.append() (where the append statements strictly consists of a string.It can be an empty string)"},
            {"role": "user", "content": condition}],
        max_tokens=150,
        temperature=0.5,
    )
    if_else_statement = response['choices'][0]['message']['content'].strip()
    # Ensure proper formatting with newlines and indentation
    formatted_if_else_statement = if_else_statement.replace(":", ":\n    ").replace(" else", "\nelse")
    return formatted_if_else_statement

# Function to split text into sentences
def split_into_sentences(text):
    # Simple sentence splitter
    sentences = re.split(r'(?<=[.!?]) +', text)
    return sentences

# Apply the function to the 'Content' column
conditions = []

for content in df['Content']:
    if isinstance(content, str):
        sentences = split_into_sentences(content)
        for sentence in sentences:
            condition = generate_conditions(sentence)
            if_else_statement = generate_if_else_statement(condition)
            conditions.append({'Sentence': sentence, 'GeneratedCondition': condition, 'IfElseStatement': if_else_statement})

# Convert the list of dictionaries to a DataFrame
conditions_df = pd.DataFrame(conditions)

# Display the DataFrame with generated conditions and if-else statements
conditions_df.head(20)

,Sentence,GeneratedCondition,IfElseStatement
0,,"""If you have a clean driving record with no hi...","```python\nif insurance_policy == ""clean drivi..."
1,2.,"""If you have a clean driving record with no re...","if insurance_policy == ""clean driving record"" ..."
2,Scheme Established Date: 1 May 1974 3.,If the scheme was established after 1 May 1974...,"if insurance_policy == ""established after 1 Ma..."
3,Contracted Out: Yes 4.,"If ""Contracted Out"" is equal to ""Yes 4"", then ...","if insurance_policy == ""Contracted Out"" and in..."
4,Contracted Out Ended Date: 30 April 2003 5.,If the contracted out ended date is on or afte...,"```python\nif contracted_out_ended_date >= ""20..."
5,Post 5 April 1997 Basis: Reference Scheme Test 6.,If the insurance policy is requested based on ...,"```python\nif insurance_policy == ""post-5 Apri..."
6,Equalisation Date: Scheme 1 - 1 November 1993 7.,If the insurance policy is being quoted for a ...,"```python\nif insurance_policy >= ""1993-11-01""..."
7,Payment Terms: - Frequency Of Payment: Mont...,If the frequency of payment is monthly and the...,"```python\nif frequency_of_payment == ""monthly..."
8,Retirement Dates and Definitions: - Normal ...,"To receive an insurance quote, the applicant m...","```python\nif insurance_policy == ""life"" and (..."
9,Pension Revaluation in Deferment: - Pre 6/4...,If the pension revaluation in deferment includ...,"```python\nif insurance_policy == ""Pre 6/4/198..."


In [19]:
formatted_conditions_file_path = '/content/formatted_conditions.txt'
with open(formatted_conditions_file_path, 'w') as file:
    for condition in conditions_df['IfElseStatement']:
        file.write(condition + "\n\n")

print(f"Formatted conditions have been written to {formatted_conditions_file_path}")

Formatted conditions have been written to /content/formatted_conditions.txt


In [20]:
# Function to format the if-else statement
def format_if_else_statement(statement):
    # Remove ''' python ''' from the statement
    statement = statement.replace("```python\n","")
    statement = statement.replace( '```\n','')

    # Check if the statement contains " else: "
    if " else: " in statement:
        # Split the statement by the if-else structure
        parts = statement.split(" else: ")
        if_part = parts[0]
        else_part = parts[1]

        # Format the if part
        formatted_if_part = if_part.replace(":", ":\n    ")

        # Combine with the else part
        formatted_statement = f"{formatted_if_part}\nelse:\n    {else_part}"

        return formatted_statement
    else:
        # Return the statement as-is if the structure is not found
        return statement

# Path to the input text file
input_file_path = '/content/formatted_conditions.txt'

# Path to the output text file
output_file_path = '/content/formatted_conditions1.txt'

# Read the content of the input text file
with open(input_file_path, 'r') as file:
    lines = file.readlines()

# Process each line
formatted_lines = [format_if_else_statement(line) for line in lines]



# Write the formatted content to the output text file
with open(output_file_path, 'w') as file:
    file.writelines(formatted_lines)

print(f"Formatted conditions have been written to {output_file_path}")


Formatted conditions have been written to /content/formatted_conditions1.txt


In [ ]:
"I have the predefined fields like Age, "

In [ ]:
#Agent1 - Input the document which is provided by the insurance company.. Out put should be rules/summarization
#Agent2 - Input the rules generated and predefined fields. Get a mapping from the rules and the fields. Output is new rules from the predifined fields
#Agent3 - From the new rules, give the data and calculate the price of the premium
#Finally get what is the total price of the premium. 
